# pipeline

## item pipeline
- 项目管道，当item生成后，会自动送到item pipeline进行处理
- 常用操作
    - 清理HTML数据
    - 验证爬取数据，检查爬取字段
    - 查重并丢弃重复内容
    - 将爬取结果保存到数据库

### 实现方法
- 定义一个类并实现 process_item()
- 启用item pipeline后，会自动调用这个方法
- 该方法必须返回包含数据的字典或item对象或抛出dropitem异常

In [ ]:
'''
pipelines.py文件中：对数据的处理
'''
from scrapy.exceptions import DropItem

class TextPipeline(object):
    def __init__(self):
        self.limit = 50
    
    def process_item(self,item,spider):
        if item['text']:
            if len(item['text']) > self.limit:
                item['text'] = item['text'][0:self.limit].rstrip() + '...'
            return item
        else:
            return DropItem('Missing Text')

In [ ]:
'''
pipelines.py文件中：存储到mongodb数据库中
'''
import pymongo
class MongoPipeline(object):
    def __init__(self,mongo_uri,mongo_db):
        self.mongo_uri = mongo_url
        self.mongo_db = mongo_db
    
    @classmethod
    '''
    类方法：通过crawler可以拿到全局配置的每个配置信息，即settings.py中的配置内容
    '''
    def form_crawler(cls,crawler):
        return cls(
            mongo_uri = crawler.settings.get('MONGO_URI'),
            mongo_db = crawler.setting.get('MONGO_DB')
        )
    '''
    当spider开启时，会被调用
    '''
    def open_spider(self,spider):
        self.client = pymongo.MongoClient(self.mongo_uri)
        self.db = self.client[self.mongo_db]
    '''
    数据的插入
    '''
    def process_item(self,item,spider):
        name = item.__class__.__name__
        self.db[name].insert(dict(item))
        return item

    '''
    当spider关闭时，调用该方法
    '''
    def close_spider(self,spider):
        self.client.close()